### Functions


function for converding CV to string so i can implement it in LLM

In [7]:
import pdfplumber
from PIL import Image
import pytesseract

def pdf_to_text(pdf_path: str) -> str:
    """
    Converts a PDF (text or scanned) to plain text.
    
    Args:
        pdf_path (str): Path to the PDF file.
        
    Returns:
        str: Extracted text from the PDF.
    """
    text = ""

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                # Try text extraction first
                page_text = page.extract_text()
                
                if page_text and page_text.strip():
                    text += page_text + "\n"
                else:
                    # Fallback to OCR for scanned/image PDFs
                    # Convert PDF page to image
                    page_image = page.to_image(resolution=300).original
                    ocr_text = pytesseract.image_to_string(page_image, lang='eng')
                    if ocr_text.strip():
                        text += ocr_text + "\n"
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {e}")
    
    return text.strip()

In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI


In [6]:
load_dotenv()  # Loads variables from .env

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [10]:
def call_llm(master_cv: str, job: Dict) -> str:
    prompt = f"""
SYSTEM INSTRUCTIONS:

You are an expert resume and cover letter writer. You are given:
1. A candidate's CV
2. A job object (title, company, description, location, etc.)

Your task:
- Rewrite the CV specifically for this job.
- Write a tailored cover letter (max 350 words).

Rules:
- Do NOT invent skills, experience, or metrics.
- Extract seniority level, workplace type (remote/hybrid/onsite), and required skills from the job description.
- Mirror terminology, highlight relevant projects, optimize for ATS.
- Adjust tone based on company description.
- Keep formatting professional and concise.

Candidate CV:
{master_cv}

Job Information:
Company: {job['company_name']}
Job Title: {job['job_title']}
Location: {job['location']}
Company Description: {job['company_description']}
Job Description: {job['job_description']}
Job URL: {job['job_url']}

OUTPUT FORMAT:

====================================================
JOB: {job['company_name']} – {job['job_title']}
====================================================

----- TAILORED CV -----
[Place rewritten CV here]

----- COVER LETTER -----
[Place cover letter here]

End output. Do not include explanations or extra text.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert resume and cover letter writer."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content

# -----------------------------
# MULTI-JOB PROCESSING FUNCTION
# -----------------------------
def process_jobs(master_cv: str, jobs: List[Dict]) -> Dict:
    """
    Loops through all jobs, calls the LLM individually, and stores outputs.
    Returns a dict mapping job_url -> LLM output.
    """
    outputs = {}
    for job in jobs:
        result = call_llm(master_cv, job)
        outputs[job['job_url']] = result
    return outputs

Expample call functions 

In [11]:
pdf_file = "CV 0.1.pdf"
cv_text = pdf_to_text(pdf_file)

# Print first 500 characters
print(cv_text[:500])

RESENT PROFESSIONAL
EXPERIENCE
Installation and scheduling Success
coordinator
Agilent Technologies • Aug. 2022- Jan.2026
Coordinated cross-functional biotech services at
Agilent, managing workflows, project delivery, and
client satisfaction in a global lab environment.
Delivered multilingual (4 languages) internal and
client communication while operating SAP
(CRM/MRS) and Microsoft Office to streamline
operations.
Costumer support and sales agent
Roomhousing Barcelona • May.2021 - Jul.2022
Gail


In [12]:
all_jobs =[{'country': 'spain',
  'job_title': 'Data Scientist Intern - Data Platform & AI',
  'company_name': 'Zurich Insurance',
  'company_url': 'https://ch.linkedin.com/company/zurich-insurance-company-ltd?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'location': 'Barcelona, Catalonia, Spain',
  'benefit': '',
  'posted': '',
  'company_description': "We Are Waiting for You\nHi there future intern!\nI am\nAlba Marin\nthe recruiter for this position (and hopefully, your future colleague!).\nFirst, a little bit about what I like the most about working at\nZurich Technology Delivery Center\n: The environment here is incredibly collaborative and innovative. Our tech-driven mindset fosters continuous learning and growth, creating an atmosphere where everyone can thrive. Now, I can't wait to share this exciting journey with you at\nZurich\n!\nLet me take a wild guess: you’ve probably sifted through countless job offers today, and they all seem to blur together. Sound familiar? Well, here’s the scoop: At Zurich, we might not have all the answers (and we’re not striving for perfection!), but we’ve got something special. Curious? Keep reading!\nFirst of all, THANK YOU for considering our company. We understand that finding the right opportunity can be tough, but I’ll try to make it easier.\nWho Are We?\nLet me introduce us. You’re reading about\nZurich Insurance Group\n, a global leader in insurance, but this role specifically pertains to our\nTechnology Delivery Center (TDC).\nEstablished in Barcelona in 2006, the TDC operates as an integral part of Zurich Insurance Group, providing cutting-edge technological solutions and support to Zurich's local and global business units.\nIn essence, Zurich TDC is where technology meets the vast reach of a multinational company. We specialize in areas such as software development, data analytics, cybersecurity, and IT infrastructure. Our primary mission is to support Zurich's strategic goals and enhance operational efficiency through innovative technology solutions.\nIf you're considering a role with us, know that you'll be joining a hub of technological excellence dedicated to driving Zurich's success on a global scale.\nWhat can you expect?\nAnd you might be thinking: This is great, what will my day-to-day be like, at TDC? I’m glad you asked – I asked myself the same question at the time.\nAs a\nData Scientist Intern – Data Platform & AI\nyou will have the opportunity to:.\nCollaborate with platform and solution engineers to design and develop new data platform features\nBuild and prototype data-driven logic for automated classification, detection, and quality assessment\nAnalyze structured and semi-structured datasets to validate feature behavior and performance\nImplement and test algorithms, rules, and heuristics to improve accuracy and reliability\nSupport integration of data science components into existing platform services and pipelines\nEvaluate results, identify edge cases, and iterate to reduce errors and improve robustness\nContribute to documentation, technical specifications, and knowledge sharing\nWork alongside experts in various technological domains.\nEnhance your skills with real-world experience\nContribute to a team that values your input and creativity\nWhat Makes You a great Fit?\nWhile we’re not seeking perfection or expecting you to save the world, we believe that if you meet some of these requirements, you’ll be a fantastic fit for our team.\nWe are waiting for you!\nCurrently pursuing or recently completed a degree in Data Science, Computer Science, AI, or a related field\nStrong Python fundamentals and ability to write clean, maintainable code\nSolid understanding of data structures, schemas, and common data formats\nBasic knowledge of machine learning, statistics, and classification concepts\nExperience with SQL and data exploration techniques\nAbility to design and reason about deterministic logic such as rules, heuristics, and pattern matching\nFamiliarity with version control and collaborative development workflows\nAnalytical mindset, attention to detail, and comfort working with ambiguous or imperfect data\nStrong communication skills and ability to work effectively with engineers\nActive student status with a Spanish University/School is mandatory.\nPassionate about a tech career.\nAvailable for at least a 6-month part-time/internship program.\nTech-savvy with a curious mind.\nEnthusiastic, capable, and eager to learn.\nA good team player with a responsible, can-do attitude.\nAnd… What Makes Us Special?\nPaid Internship, up to 30 hours weekly with a flexible schedule.\nLinkedIn and internal training opportunities.\nSpecial discounts on various brands and products, plus great banking and insurance conditions.\nA welcome pack filled with amazing gadgets.\nA dynamic workspace with tasty coffee and fruit and featuring gaming, gym, and other fun areas!\nSummer and Christmas events and different social clubs such as hiking, investment, beach volley and many more\nLooking for a place to start or further develop your career?\nThis is for you! Zurich offers a comprehensive career plan providing a great opportunity to build and enhance your career from the start. You will have the chance to work on global projects, guided by experts and leveraging innovative technologies. Learn from very well-known experts in the field, ensuring you gain valuable experience and knowledge.\nWhere Are We Located?\nBarcelona, Poblenou.\nWe Are Waiting for You.\nIt’s a match? Don’t wait any longer!\nApply\nby sending your\nCV in English\n.\nIf not, no worries! We’ll meet again in the future. Feel free to share this amazing opportunity with anyone you think would be a great fit.\nAt Zurich, we are an equal opportunity employer. We attract and retain the best-qualified individuals available, regardless of race/ethnicity, religion, gender, sexual orientation, age, or disability.\nZurich Technology Delivery Center – Your Talent, Our Strength\nYou are the heart & soul of Zurich!\nAt Zurich, we like to think outside the box and challenge the status quo. We take an optimistic approach by focusing on the positives and constantly asking What can go right?\nWe highly value the experience and know-how of our employees and offer a wide range of opportunities across business areas to encourage you to apply for new opportunities within Zurich when you are ready for your next career step.\nLet’s continue to grow together!\nLocation(s): ES - Barcelona\nSchedule: Part Time\nRecruiter name: Alba Marin Sola",
  'job_url': 'https://es.linkedin.com/jobs/view/data-scientist-intern-data-platform-ai-at-zurich-insurance-4362010365?position=1&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=NmjmiVP8EcoWcRO5ILV6FA%3D%3D',
  'job_description': "We Are Waiting for You\nHi there future intern!\nI am\nAlba Marin\nthe recruiter for this position (and hopefully, your future colleague!).\nFirst, a little bit about what I like the most about working at\nZurich Technology Delivery Center\n: The environment here is incredibly collaborative and innovative. Our tech-driven mindset fosters continuous learning and growth, creating an atmosphere where everyone can thrive. Now, I can't wait to share this exciting journey with you at\nZurich\n!\nLet me take a wild guess: you’ve probably sifted through countless job offers today, and they all seem to blur together. Sound familiar? Well, here’s the scoop: At Zurich, we might not have all the answers (and we’re not striving for perfection!), but we’ve got something special. Curious? Keep reading!\nFirst of all, THANK YOU for considering our company. We understand that finding the right opportunity can be tough, but I’ll try to make it easier.\nWho Are We?\nLet me introduce us. You’re reading about\nZurich Insurance Group\n, a global leader in insurance, but this role specifically pertains to our\nTechnology Delivery Center (TDC).\nEstablished in Barcelona in 2006, the TDC operates as an integral part of Zurich Insurance Group, providing cutting-edge technological solutions and support to Zurich's local and global business units.\nIn essence, Zurich TDC is where technology meets the vast reach of a multinational company. We specialize in areas such as software development, data analytics, cybersecurity, and IT infrastructure. Our primary mission is to support Zurich's strategic goals and enhance operational efficiency through innovative technology solutions.\nIf you're considering a role with us, know that you'll be joining a hub of technological excellence dedicated to driving Zurich's success on a global scale.\nWhat can you expect?\nAnd you might be thinking: This is great, what will my day-to-day be like, at TDC? I’m glad you asked – I asked myself the same question at the time.\nAs a\nData Scientist Intern – Data Platform & AI\nyou will have the opportunity to:.\nCollaborate with platform and solution engineers to design and develop new data platform features\nBuild and prototype data-driven logic for automated classification, detection, and quality assessment\nAnalyze structured and semi-structured datasets to validate feature behavior and performance\nImplement and test algorithms, rules, and heuristics to improve accuracy and reliability\nSupport integration of data science components into existing platform services and pipelines\nEvaluate results, identify edge cases, and iterate to reduce errors and improve robustness\nContribute to documentation, technical specifications, and knowledge sharing\nWork alongside experts in various technological domains.\nEnhance your skills with real-world experience\nContribute to a team that values your input and creativity\nWhat Makes You a great Fit?\nWhile we’re not seeking perfection or expecting you to save the world, we believe that if you meet some of these requirements, you’ll be a fantastic fit for our team.\nWe are waiting for you!\nCurrently pursuing or recently completed a degree in Data Science, Computer Science, AI, or a related field\nStrong Python fundamentals and ability to write clean, maintainable code\nSolid understanding of data structures, schemas, and common data formats\nBasic knowledge of machine learning, statistics, and classification concepts\nExperience with SQL and data exploration techniques\nAbility to design and reason about deterministic logic such as rules, heuristics, and pattern matching\nFamiliarity with version control and collaborative development workflows\nAnalytical mindset, attention to detail, and comfort working with ambiguous or imperfect data\nStrong communication skills and ability to work effectively with engineers\nActive student status with a Spanish University/School is mandatory.\nPassionate about a tech career.\nAvailable for at least a 6-month part-time/internship program.\nTech-savvy with a curious mind.\nEnthusiastic, capable, and eager to learn.\nA good team player with a responsible, can-do attitude.\nAnd… What Makes Us Special?\nPaid Internship, up to 30 hours weekly with a flexible schedule.\nLinkedIn and internal training opportunities.\nSpecial discounts on various brands and products, plus great banking and insurance conditions.\nA welcome pack filled with amazing gadgets.\nA dynamic workspace with tasty coffee and fruit and featuring gaming, gym, and other fun areas!\nSummer and Christmas events and different social clubs such as hiking, investment, beach volley and many more\nLooking for a place to start or further develop your career?\nThis is for you! Zurich offers a comprehensive career plan providing a great opportunity to build and enhance your career from the start. You will have the chance to work on global projects, guided by experts and leveraging innovative technologies. Learn from very well-known experts in the field, ensuring you gain valuable experience and knowledge.\nWhere Are We Located?\nBarcelona, Poblenou.\nWe Are Waiting for You.\nIt’s a match? Don’t wait any longer!\nApply\nby sending your\nCV in English\n.\nIf not, no worries! We’ll meet again in the future. Feel free to share this amazing opportunity with anyone you think would be a great fit.\nAt Zurich, we are an equal opportunity employer. We attract and retain the best-qualified individuals available, regardless of race/ethnicity, religion, gender, sexual orientation, age, or disability.\nZurich Technology Delivery Center – Your Talent, Our Strength\nYou are the heart & soul of Zurich!\nAt Zurich, we like to think outside the box and challenge the status quo. We take an optimistic approach by focusing on the positives and constantly asking What can go right?\nWe highly value the experience and know-how of our employees and offer a wide range of opportunities across business areas to encourage you to apply for new opportunities within Zurich when you are ready for your next career step.\nLet’s continue to grow together!\nLocation(s): ES - Barcelona\nSchedule: Part Time\nRecruiter name: Alba Marin Sola\nShow more\nShow less"},
{'country': 'spain',
  'job_title': 'Programa Talento Joven - Data Science',
  'company_name': 'Evolve',
  'company_url': 'https://es.linkedin.com/school/evolve-es/?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'location': 'Spain',
  'benefit': 'Be an early applicant',
  'posted': '',
  'company_description': 'Descripción de la oferta\nEn nuestro Programa Talento Joven buscamos personas recién tituladas en Formación Profesional de informática, análisis de datos o programación, con interés en adquirir experiencia y conocimientos prácticos en el campo del Data Science.\nFunciones:\nParticipar en un programa formativo de 30 semanas enfocado en habilidades técnicas y prácticas de análisis de datos\nTrabajar sobre proyectos reales y simulaciones del entorno empresarial, incluyendo retos de predicción, análisis exploratorio y visualización de datos\nColaborar y aprender directamente de profesionales en activo que te acompañarán durante tu evolución\nPracticar con herramientas como Python, SQL, Pandas, Power BI, Jupyter Notebooks y Scikit-learn\nAplicar procedimientos y buenas prácticas de análisis en entornos controlados de laboratorio\nLo que ofrecemos:\nPlan formativo diseñado y actualizado según las necesidades reales del mercado laboral en data\nFormación práctica, en grupo reducido, orientada al trabajo que se realiza en equipos de data science\nMentoring personalizado con especialistas del sector\nPosibilidad real de entrevista para ofertas de empleo al finalizar el programa en empresas colaboradoras\nRed de contacto con profesionales y exalumnos del programa\nRequisitos:\nCiclo Formativo de Grado Medio o Superior relacionado con informática, análisis de datos o programación\nInterés por el análisis de datos y motivación para afrontar retos prácticos\nCapacidad para trabajar en equipo, comunicarte y aprender en un entorno técnico\nTipo de puesto:\nFormativo/Prácticas\nUbicación:\nModalidad híbrida (presencial y online)\nDuración:\n30 semanas\nIncorporación:\nAbril 2026',
  'job_url': 'https://es.linkedin.com/jobs/view/programa-talento-joven-data-science-at-evolve-4371024018?position=2&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=YGFP9eTxYU%2Bgf0IRXNGj8Q%3D%3D',
  'job_description': 'Descripción de la oferta\nEn nuestro Programa Talento Joven buscamos personas recién tituladas en Formación Profesional de informática, análisis de datos o programación, con interés en adquirir experiencia y conocimientos prácticos en el campo del Data Science.\nFunciones:\nParticipar en un programa formativo de 30 semanas enfocado en habilidades técnicas y prácticas de análisis de datos\nTrabajar sobre proyectos reales y simulaciones del entorno empresarial, incluyendo retos de predicción, análisis exploratorio y visualización de datos\nColaborar y aprender directamente de profesionales en activo que te acompañarán durante tu evolución\nPracticar con herramientas como Python, SQL, Pandas, Power BI, Jupyter Notebooks y Scikit-learn\nAplicar procedimientos y buenas prácticas de análisis en entornos controlados de laboratorio\nLo que ofrecemos:\nPlan formativo diseñado y actualizado según las necesidades reales del mercado laboral en data\nFormación práctica, en grupo reducido, orientada al trabajo que se realiza en equipos de data science\nMentoring personalizado con especialistas del sector\nPosibilidad real de entrevista para ofertas de empleo al finalizar el programa en empresas colaboradoras\nRed de contacto con profesionales y exalumnos del programa\nRequisitos:\nCiclo Formativo de Grado Medio o Superior relacionado con informática, análisis de datos o programación\nInterés por el análisis de datos y motivación para afrontar retos prácticos\nCapacidad para trabajar en equipo, comunicarte y aprender en un entorno técnico\nTipo de puesto:\nFormativo/Prácticas\nUbicación:\nModalidad híbrida (presencial y online)\nDuración:\n30 semanas\nIncorporación:\nAbril 2026\nShow more\nShow less'},
 {'country': 'spain',
  'job_title': 'Data Engineer / Data Scientist',
  'company_name': 'Inetum',
  'company_url': 'https://fr.linkedin.com/company/inetum?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'location': 'Madrid, Community of Madrid, Spain',
  'benefit': '',
  'posted': '',
  'company_description': 'Descripción de la empresa\nInetum es líder europeo en servicios digitales. El equipo de 28.000 consultores y especialistas de Inetum se esfuerza cada día por lograr un impacto digital para las empresas, las entidades del sector público y la sociedad. Las soluciones de Inetum tienen como objetivo contribuir al rendimiento y la innovación de sus clientes, así como al bien común.\nPresente en 19 países con una densa red de centros, Inetum se asocia con los principales fabricantes de software para afrontar los retos de la transformación digital desde la proximidad y flexibilidad.\nDescripción del empleo\nEn Inetum buscamos incorporar un/a Data Engineer/Data Scientist, para un equipo de trabajo, en uno de nuestros clientes mas estratégicos, para nuestra oficina de Madrid.\nObjetivo:\nGobernanza, estructuración, limpieza y calidad del dato; preparación de datasets para entrenamiento; pipelines robustos de ingestión y transformación.\nResponsabilidades Principales\nAdquisición y normalización de datos desde múltiples fuentes.\nDiseño de data lakes, feature stores y pipelines ETL/ELT.\nCreación de datasets curados y auditables (train/val/test).\nAutomatización.\nKPI de calidad del dato; apoyo en experimentación y reproducibilidad.\nGeneración de datos sintéticos cuando sea necesario.\nRequisitos\nNecesitamos Que Cuentes Con Los Siguientes Requisitos\nPython, SQL, BBDD noSQL (neo4j, MongoDB...).\nExperiencia en anotación y versionado de datos.\nEntendimiento suficiente de ML para preparar datos óptimos.\nDeseable, conocimiento de Frontend (Angular) y Backend (.net), Jira, Git\n¿Eres la persona que buscamos? Inscríbete en nuestra oferta y haznos llegar tu perfil completo. ¡Estamos deseando contar contigo!\nInformación adicional\n¿QUÉ OFRECEMOS?\nContrato indefinido.\nFlexibilidad horaria.\nPlan de carrera: ofrecemos una atractiva carrera profesional en función de la experiencia y potencial personal dentro de una compañía en continua evolución y con un sólido crecimiento\nPlan de formación en competencias tecnológicas de acuerdo con las exigencias de los proyectos y clientes.\nRetribución flexible: te ofrecemos una retribución a la carta pudiendo elegir diferentes productos y modelar tú mismo cómo distribuirlos: seguro de salud, tickets de comida, guardería, tarjeta transporte, etc.',
  'job_url': 'https://es.linkedin.com/jobs/view/data-engineer-data-scientist-at-inetum-4370983244?position=3&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=QrTduYPtg0HSD44YORrdew%3D%3D',
  'job_description': 'Descripción de la empresa\nInetum es líder europeo en servicios digitales. El equipo de 28.000 consultores y especialistas de Inetum se esfuerza cada día por lograr un impacto digital para las empresas, las entidades del sector público y la sociedad. Las soluciones de Inetum tienen como objetivo contribuir al rendimiento y la innovación de sus clientes, así como al bien común.\nPresente en 19 países con una densa red de centros, Inetum se asocia con los principales fabricantes de software para afrontar los retos de la transformación digital desde la proximidad y flexibilidad.\nDescripción del empleo\nEn Inetum buscamos incorporar un/a Data Engineer/Data Scientist, para un equipo de trabajo, en uno de nuestros clientes mas estratégicos, para nuestra oficina de Madrid.\nObjetivo:\nGobernanza, estructuración, limpieza y calidad del dato; preparación de datasets para entrenamiento; pipelines robustos de ingestión y transformación.\nResponsabilidades Principales\nAdquisición y normalización de datos desde múltiples fuentes.\nDiseño de data lakes, feature stores y pipelines ETL/ELT.\nCreación de datasets curados y auditables (train/val/test).\nAutomatización.\nKPI de calidad del dato; apoyo en experimentación y reproducibilidad.\nGeneración de datos sintéticos cuando sea necesario.\nRequisitos\nNecesitamos Que Cuentes Con Los Siguientes Requisitos\nPython, SQL, BBDD noSQL (neo4j, MongoDB...).\nExperiencia en anotación y versionado de datos.\nEntendimiento suficiente de ML para preparar datos óptimos.\nDeseable, conocimiento de Frontend (Angular) y Backend (.net), Jira, Git\n¿Eres la persona que buscamos? Inscríbete en nuestra oferta y haznos llegar tu perfil completo. ¡Estamos deseando contar contigo!\nInformación adicional\n¿QUÉ OFRECEMOS?\nContrato indefinido.\nFlexibilidad horaria.\nPlan de carrera: ofrecemos una atractiva carrera profesional en función de la experiencia y potencial personal dentro de una compañía en continua evolución y con un sólido crecimiento\nPlan de formación en competencias tecnológicas de acuerdo con las exigencias de los proyectos y clientes.\nRetribución flexible: te ofrecemos una retribución a la carta pudiendo elegir diferentes productos y modelar tú mismo cómo distribuirlos: seguro de salud, tickets de comida, guardería, tarjeta transporte, etc.\nShow more\nShow less'},
 {'country': 'spain',
  'job_title': 'Machine Learning Engineer - AI Data Trainer',
  'company_name': 'Alignerr',
  'company_url': 'https://www.linkedin.com/company/alignerr?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'location': 'Spain',
  'benefit': '',
  'posted': '',
  'company_description': 'Location: Remote\nAbout The Job\nAt Alignerr, we partner with the world’s leading AI research teams and labs to build and train cutting-edge AI models.\nThis project is about capturing how an LLM reasons and takes actions step by step to solve real tasks. Annotators write clear, structured traces that show planning, tool use, and decision making, creating data that trains models to reason more reliably in real world scenarios.\nOrganization\n: Alignerr\nPosition\n: Machine Learning Engineer - AI Data Trainer\nType\n: Hourly Contract\nCompensation\n: $50–$70 /hour\nLocation\n: Remote\nCommitment\n: 10–40 hours/week\nWhat You’ll Do\nAuthor clear, structured reasoning traces for complex technical tasks.\nDocument step-by-step planning and decision-making processes.\nCreate training data that demonstrates effective tool use in real-world logic.\nRequirements\nBackground in Machine Learning, Computer Science, or a related technical field.\nAbility to articulate complex logical steps and technical reasoning in writing.\nStrong analytical skills to evaluate and improve AI decision-making workflows.\nPreferred\nPrior experience with data annotation, data quality, or evaluation systems.\nWhy Join Us\nCompetitive pay and flexible remote work.\nCollaborate with a team working on cutting-edge AI projects.\nExposure to advanced LLMs and how they’re trained.\nFreelance perks: autonomy, flexibility, and global collaboration.\nPotential for contract extension.\nApplication Process (Takes 15-20 min)\nSubmit your resume\nComplete a short screening\nProject matching and onboarding\nPS: Our team reviews applications daily. Please complete your AI interview and application steps to be considered for this opportunity.',
  'job_url': 'https://es.linkedin.com/jobs/view/machine-learning-engineer-ai-data-trainer-at-alignerr-4370938346?position=4&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=XPUeZ5ZOuv5DiFzBYGmqKg%3D%3D',
  'job_description': 'Location: Remote\nAbout The Job\nAt Alignerr, we partner with the world’s leading AI research teams and labs to build and train cutting-edge AI models.\nThis project is about capturing how an LLM reasons and takes actions step by step to solve real tasks. Annotators write clear, structured traces that show planning, tool use, and decision making, creating data that trains models to reason more reliably in real world scenarios.\nOrganization\n: Alignerr\nPosition\n: Machine Learning Engineer - AI Data Trainer\nType\n: Hourly Contract\nCompensation\n: $50–$70 /hour\nLocation\n: Remote\nCommitment\n: 10–40 hours/week\nWhat You’ll Do\nAuthor clear, structured reasoning traces for complex technical tasks.\nDocument step-by-step planning and decision-making processes.\nCreate training data that demonstrates effective tool use in real-world logic.\nRequirements\nBackground in Machine Learning, Computer Science, or a related technical field.\nAbility to articulate complex logical steps and technical reasoning in writing.\nStrong analytical skills to evaluate and improve AI decision-making workflows.\nPreferred\nPrior experience with data annotation, data quality, or evaluation systems.\nWhy Join Us\nCompetitive pay and flexible remote work.\nCollaborate with a team working on cutting-edge AI projects.\nExposure to advanced LLMs and how they’re trained.\nFreelance perks: autonomy, flexibility, and global collaboration.\nPotential for contract extension.\nApplication Process (Takes 15-20 min)\nSubmit your resume\nComplete a short screening\nProject matching and onboarding\nPS: Our team reviews applications daily. Please complete your AI interview and application steps to be considered for this opportunity.\nShow more\nShow less'},
]

In [14]:
job_outputs = process_jobs(cv_text, all_jobs)

In [15]:
job_outputs

{'https://es.linkedin.com/jobs/view/data-scientist-intern-data-platform-ai-at-zurich-insurance-4362010365?position=1&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=NmjmiVP8EcoWcRO5ILV6FA%3D%3D': "====================================================\nJOB: Zurich Insurance – Data Scientist Intern - Data Platform & AI\n====================================================\n\n----- TAILORED CV -----\n**Gail Marechal**  \ngail.marechal@hotmail.be  \n+34 621 264 417  \nBarcelona, Catalonia, Spain  \nGithub: gail-mar  \n\n**EDUCATION**  \nTechnical Master in Data Science and AI (Present)  \nIronhacks  \n\nBachelor in Tourism and Hospitality Management (2024)  \nUniversity of Lleida  \n\n**PROFESSIONAL EXPERIENCE**  \n\n**Installation and Scheduling Success Coordinator**  \nAgilent Technologies • Aug. 2022 - Jan. 2026  \n- Managed workflows and project delivery in a global lab environment, enhancing operational efficiency.  \n- Delivered multilingual (4 languages) internal and client c

In [16]:
for k,v in job_outputs.items():
    print(k)
    print(v)
    print("\n")

https://es.linkedin.com/jobs/view/data-scientist-intern-data-platform-ai-at-zurich-insurance-4362010365?position=1&pageNum=0&refId=orJxOh27fmOHa2i8hFGveg%3D%3D&trackingId=NmjmiVP8EcoWcRO5ILV6FA%3D%3D
JOB: Zurich Insurance – Data Scientist Intern - Data Platform & AI

----- TAILORED CV -----
**Gail Marechal**  
gail.marechal@hotmail.be  
+34 621 264 417  
Barcelona, Catalonia, Spain  
Github: gail-mar  

**EDUCATION**  
Technical Master in Data Science and AI (Present)  
Ironhacks  

Bachelor in Tourism and Hospitality Management (2024)  
University of Lleida  

**PROFESSIONAL EXPERIENCE**  

**Installation and Scheduling Success Coordinator**  
Agilent Technologies • Aug. 2022 - Jan. 2026  
- Managed workflows and project delivery in a global lab environment, enhancing operational efficiency.  
- Delivered multilingual (4 languages) internal and client communications while utilizing SAP (CRM/MRS).  
- Collaborated cross-functionally to ensure client satisfaction and streamlined operati